In [1]:
from langchain_openai import ChatOpenAI
import os
deepseek_key=os.getenv("DEEPSEEK_API_KEY")
llm=ChatOpenAI(temperature=0.95,model="deepseek-chat",api_key=deepseek_key,base_url="https://api.deepseek.com")

In [2]:
import requests
import json
import os

# 获取 API Key
newsapi_key = os.getenv("NewsAPI")
if not newsapi_key:
    raise ValueError("❌ API Key 未找到，请检查环境变量 `NewsAPI`")

# 定义 URL 列表
categories = [
    "农林牧渔", "基础化工", "钢铁", "有色金属", "电子", "家用电器", "食品饮料", "纺织服饰",
    "轻工制造", "医药生物", "公用事业", "交通运输", "房地产", "商贸零售", "社会服务",
    "综合", "建筑材料", "建筑装饰", "电力设备", "国防军工", "计算机", "传媒", "通信",
    "银行", "非银金融", "汽车", "机械设备", "煤炭", "石油石化", "环保", "美容护理"
]

url_list = [f'https://newsapi.org/v2/everything?q={category}&apiKey={newsapi_key}' for category in categories]

# **数据存储文件**
output_file = r"C:\Users\lenovo\Documents\industry news\news_data.json"

def fetch_news():
    """获取新闻数据并保存"""
    content = []
    
    for url, category in zip(url_list, categories):
        try:
            response = requests.get(url)
            response.raise_for_status()  # 如果状态码不是 200，会抛出 HTTPError 异常
            data = response.json()
            
            # 确保 articles 存在
            articles = data.get("articles", [])
            if not articles:
                print(f"⚠️ {category} 没有新闻数据")
                continue
            
            for article in articles:
                content.append({
                    "category": category,
                    "title": article.get("title", "无标题"),
                    "description": article.get("description", "无描述"),
                    "url": article.get("url", "#"),
                    "published_at": article.get("publishedAt", "未知时间")
                })
            
        except requests.exceptions.RequestException as e:
            print(f"❌ 请求失败: {category}, 错误信息: {e}")
    
    # **保存数据**
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(content, f, ensure_ascii=False, indent=4)
    
    print(f"✅ 新闻数据已保存到 {output_file}")

def load_news():
    """从 JSON 加载已保存的数据"""
    if not os.path.exists(output_file):
        print("⚠️ 未找到缓存数据，开始抓取新闻...")
        fetch_news()
    
    with open(output_file, "r", encoding="utf-8") as f:
        content = json.load(f)
    
    print(f"📄 成功加载 {len(content)} 条新闻")
    return content

news_data = load_news()
print(news_data[:3])  # 只打印前 3 条新闻，防止控制台输出过多


📄 成功加载 2370 条新闻
[{'category': '农林牧渔', 'title': '政府工作报告', 'description': '各位代表：\u3000\u3000现在，我代表国务院，向大会报告政府工作，请予审议，并请全国政协委员提出意见。\u3000\u3000一、2024年工作回顾\u3000\u3000过去一年，我国发展历程很不平凡。党的二十届三中全会胜利召开，对进一步全面深化改革、推进中国式现代化作出部署。我们隆重庆祝中华人民共和国成立75周年，极大激发了全国各族人民的爱国热情和奋斗精神。一年来，面对外部压力加大、内部困难增多的复杂严峻形势，在以习近平同志为核心的党中央坚强领导下，全国各族人民砥砺奋进、攻坚克难，经济运行总体平稳、稳中有进，全年经济社会发展主要目标任务顺利完成，高质量发展扎实…', 'url': 'http://politics.people.com.cn/n1/2025/0313/c1001-40437762.html', 'published_at': '2025-03-12T22:25:28Z'}, {'category': '农林牧渔', 'title': '关于2024年国民经济和社会发展计划执行情况与2025年国民经济和社会发展计划草案的报告', 'description': '新华社北京3月13日电 关于2024年国民经济和社会发展计划执行情况与2025年国民经济和社会发展计划草案的报告 ——2025年3月5日在第十四届全国人民代表大会第三次会议上 国家发展和改革委员会 各位代表： 受国务院委托，现将2024年国民经济和社会发展计划执行情况与2025年国民经济和社会发展计划草案提请十四届全国人大三次会议审查，并请全国政协各位委员提出意见。 一、2024年国民经济和社会发展计划执行情况 2024年是中华人民共和国成立75周年，是实现“十四五”规划目标任务的关键一年。一年来，面对外部压力加…', 'url': 'http://politics.people.com.cn/n1/2025/0313/c1001-40438585.html', 'published_at': '2025-03-13T00:00:00Z'}, {'category': '农林牧渔', 'title': '关于二〇二四年国民经济和社会发展计划

In [31]:
import os
from tqdm import tqdm  # 新增导入
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# 指定本地的 Embedding 模型路径
embeddings_path = "C:\\Users\\lenovo\\bge-large-zh-v1.5"
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path, model_kwargs=model_kwargs)

# 指定索引存储路径
save_path = r"C:\Users\lenovo\Documents\industry news"

if os.path.exists(os.path.join(save_path, "index.faiss")) and os.path.exists(os.path.join(save_path, "index.pkl")):
    print("检测到本地索引，正在安全加载...")
    try:
        # 添加安全反序列化参数
        vectorstore = FAISS.load_local(
            save_path, 
            embeddings,
            allow_dangerous_deserialization=True  # 关键参数
        )
        print("索引加载成功")
    except Exception as e:
        print(f"索引加载失败: {str(e)}")
        # 可添加自动重建索引的逻辑
else:
    print("未找到本地索引，正在创建新的索引...")

    if isinstance(news_data, list) and isinstance(news_data[0], dict):
        news_data = [item["description"] for item in news_data if "description" in item]
    
    # 确保数据是字符串列表
    assert all(isinstance(text, str) for text in news_data), "news_data 必须是 List[str]"
    
    # 创建 FAISS 向量存储（带进度条）
    batch_size = 100  # 每批处理100个文档
    try:
        # 初始化第一个批次
        first_batch = news_data[:batch_size]
        vectorstore = FAISS.from_texts(first_batch, embeddings)
        
        # 创建进度条
        with tqdm(total=len(news_data), desc="创建索引进度", unit="doc") as pbar:
            pbar.update(len(first_batch))
            
            # 分批次处理剩余文档
            for i in range(batch_size, len(news_data), batch_size):
                batch = news_data[i:i+batch_size]
                vectorstore.add_texts(batch)
                pbar.update(len(batch))
                
    except Exception as e:
        print(f"创建索引时发生错误: {str(e)}")
        raise
    
    # 保存索引到本地
    vectorstore.save_local(save_path)
    print("\n新索引已创建并保存到:", save_path)

retriever = vectorstore.as_retriever()

result = retriever.get_relevant_documents("电力行业怎么样")

# 打印查询结果
print("\n查询结果:")
for doc in result:
    print(doc.page_content)

2025-03-25 16:02:31,943 [INFO] Use pytorch device_name: cpu
2025-03-25 16:02:31,946 [INFO] Load pretrained SentenceTransformer: C:\Users\lenovo\bge-large-zh-v1.5
检测到本地索引，正在安全加载...
索引加载成功

查询结果:
IT之家 2 月 18 日消息，第十五届中国国际清洁能源博览会（CEEC 2025）将于 2025 年 03 月 26 日 - 28 日在北京国家会议中心举行，期间将举办 30 多场重磅专题会议、40 多场新品发布会，以及国际交流活动等。在今日的新闻发布会上，中国电力企业联合会副秘书长刘永东在会上从四个方面介绍了我国能源电力行业的最新发展情况。刘永东表示，截至 2024 年底，我国可再生能源装机达到 18.89 亿千瓦，同比增长 25%，约占我国总装机的 56%，其中水电装机 4.36 亿千瓦，风电装机 5.21…
编者按：“城镇调查失业率5.5%左右，城镇新增就业1200万人以上。”3月5日提请审议的政府工作报告（以下简称“政府工作报告”）明确今年发展主要预期目标。 就业是每年全国两会最受关注的话题之一。两会前夕，“财米油盐”围绕就业话题进行街头采访，发现当前就业市场似乎遇到一面“空气墙”：求职的同学们吐槽“就业难”，而雇主们则抱怨“用工荒”。如何破解这一矛盾现状？“财米油盐”在全国两会向代表委员们寻求解答。 
 showPlayer({id:"/pvservice/xml/2025/3/7/04b7fdd0-a5cc-…
编者按：“城镇调查失业率5.5%左右，城镇新增就业1200万人以上。”3月5日提请审议的政府工作报告（以下简称“政府工作报告”）明确今年发展主要预期目标。 就业是每年全国两会最受关注的话题之一。两会前夕，“财米油盐”围绕就业话题进行街头采访，发现当前就业市场似乎遇到一面“空气墙”：求职的同学们吐槽“就业难”，而雇主们则抱怨“用工荒”。如何破解这一矛盾现状？“财米油盐”在全国两会向代表委员们寻求解答。 
 showPlayer({id:"/pvservice/xml/2025/3/7/04b7fdd0-a5cc-…
据“四川观察”官微，日前，格力电器董事长兼总裁董明

In [18]:
from datetime import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
import os
import undetected_chromedriver as uc

site_industry_report = r'http://data.eastmoney.com/report/industry.jshtml'

file_industry_report_link = r'D:\code\python\report down url\ind report url 2019-11-08 22-53-41.txt'

def scrape_industry(start_page=1, scrape_page=2, save_file=file_industry_report_link):  # 行业研报
    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    options.add_argument('--start-maximized')
    service = Service(r'C:\Users\lenovo\chromedriver-win64\chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=options)
    browser.get(site_industry_report)  # 打开网页
    time.sleep(2)
    if scrape_page == None:
        last_page_element = WebDriverWait(browser, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='pagerbox']/a[last()-1]"))
        )
        total_page = int(last_page_element.get_attribute("data-page"))
        print(total_page)
    else:
        total_page = scrape_page

    out = []
    with open(save_file, 'a+') as f:
        for page in range(start_page, total_page + 1):
            WebDriverWait(browser, 5).until(lambda x: x.find_element(By.XPATH, "//input[@id='gotopageindex']")).clear()
            browser.find_element(By.XPATH, "//input[@id='gotopageindex']").send_keys(page)
            submit_button = browser.find_element(By.CSS_SELECTOR, 'input.btn[value="确定"]')
            browser.execute_script("arguments[0].click();", submit_button)
            time.sleep(2)

            page_sourece = browser.page_source
            out.append(page_sourece)

            res_link_list = re.findall('infocode=(.*?)"', page_sourece)
            res_link_list2 = ["http://data.eastmoney.com/report/zw_industry.jshtml?infocode=%s\n" % i for i in
                              res_link_list]
            f.writelines(res_link_list2)
            strTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            print('行业 第%s页 采集数目%s  %s' % (page, len(res_link_list2), strTime))
            if len(res_link_list) == 50 and page != total_page:
                pass
            else:
                print('提示：第%s页 采集数目%s [数目不足] %s' % (page, len(res_link_list2), strTime))
    f.close()
    browser.quit()
    return out

def down_industry(start_line=1, save_file=file_industry_report_link):
    report_lis = pd.Series(list(open(save_file))).dropna()
    browsered = r'C:\Users\lenovo\Documents\industry browsered.txt'
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument("--disable-blink-features=AutomationControlled")
    service = Service(r'C:\Users\lenovo\chromedriver-win64\chromedriver.exe')
    browser2 = webdriver.Chrome(service=service, options=options)

    download_path = r"C:\Users\lenovo\Documents\industry report"
    os.makedirs(download_path, exist_ok=True)

    # 配置 Selenium 允许下载 PDF
    options1 = uc.ChromeOptions()
    prefs = {
        "download.default_directory": download_path,  # 指定下载文件夹
        "plugins.always_open_pdf_externally": True,  # 直接下载 PDF 而不是打开
    }
    options1.add_experimental_option("prefs", prefs)

    # 启动浏览器
    driver = uc.Chrome(options=options1)
    for i in range(start_line, len(report_lis)):
        try:
            strTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            link = report_lis[i].replace('\n', '')
            print('%s [行业] 开始于%s' % (i, start_line))
            if 1 == 2:
                print(i, '[已浏览] %s' % strTime)
            else:
                browser2.get(link)
                time.sleep(2)
                page_sourece = browser2.page_source
                with open(browsered, 'a+') as f3:
                    f3.write(link)
                match_pdf = re.findall('"attach_url":"(.*?)"', page_sourece)
                if len(match_pdf) == 0:
                    print(i, '[无下载地址] %s' % strTime)
                    print(page_sourece)
                else:
                    url=match_pdf[0]
                    # 访问 PDF 生成页面
                    driver.get(url)
                    print(i, '[完成下载]  %s' % (strTime))
        except Exception as e:
            print(e)
    #等待下载完成
    time.sleep(30)
    driver.quit()
    browser2.quit()


def daily_task_ind(scrape_page=4, start_line=0):
    datetime_str = datetime.now().strftime(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
    fn_industry = r'C:\Users\lenovo\Documents\report down url\ind report url %s.txt' % datetime_str
    print(fn_industry)
    scrape_industry(scrape_page=scrape_page, save_file=fn_industry)
    down_industry(save_file=fn_industry, start_line=start_line)


daily_task_ind(scrape_page=50)

C:\Users\lenovo\Documents\report down url\ind report url 2025-03-18 11-18-15.txt
行业 第1页 采集数目50  2025-03-18 11:18:23
行业 第2页 采集数目50  2025-03-18 11:18:25
行业 第3页 采集数目50  2025-03-18 11:18:28
行业 第4页 采集数目50  2025-03-18 11:18:30
行业 第5页 采集数目50  2025-03-18 11:18:32
行业 第6页 采集数目50  2025-03-18 11:18:34
行业 第7页 采集数目50  2025-03-18 11:18:36
行业 第8页 采集数目50  2025-03-18 11:18:39
行业 第9页 采集数目50  2025-03-18 11:18:41
行业 第10页 采集数目50  2025-03-18 11:18:43
行业 第11页 采集数目50  2025-03-18 11:18:45
行业 第12页 采集数目50  2025-03-18 11:18:47
行业 第13页 采集数目50  2025-03-18 11:18:49
行业 第14页 采集数目50  2025-03-18 11:18:52
行业 第15页 采集数目50  2025-03-18 11:18:54
行业 第16页 采集数目50  2025-03-18 11:18:56
行业 第17页 采集数目50  2025-03-18 11:18:58
行业 第18页 采集数目50  2025-03-18 11:19:00
行业 第19页 采集数目50  2025-03-18 11:19:02
行业 第20页 采集数目50  2025-03-18 11:19:05
行业 第21页 采集数目50  2025-03-18 11:19:07
行业 第22页 采集数目50  2025-03-18 11:19:09
行业 第23页 采集数目50  2025-03-18 11:19:11
行业 第24页 采集数目50  2025-03-18 11:19:13
行业 第25页 采集数目50  2025-03-18 11:19:15
行业 第26页 采集数目50  2025-03-18 1

In [5]:
import os
import sys  
import time
import logging
import pickle
from tqdm import tqdm
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from logging import StreamHandler

class UniversalStreamHandler(StreamHandler):
    def __init__(self):
        self._is_jupyter = 'ipykernel' in sys.modules
        stream = self._get_compatible_stream()
        super().__init__(stream)
    
    def _get_compatible_stream(self):
        """智能获取兼容输出流"""
        if self._is_jupyter:
            return sys.stdout
            
        try:
            fd = sys.stdout.fileno()
            return open(fd, 
                       mode='w', 
                       encoding='utf-8',
                       errors='replace',
                       buffering=1)
        except (UnsupportedOperation, AttributeError):
            return sys.stdout

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler('app.log', encoding='utf-8'),
        UniversalStreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# 新增缓存相关函数
def get_cache_path(folder_path: str) -> str:
    """获取缓存文件路径"""
    cache_dir = os.path.join(folder_path, ".cache")
    os.makedirs(cache_dir, exist_ok=True)
    return os.path.join(cache_dir, "docs_cache.pkl")

def get_current_metadata(pdf_files: list) -> dict:
    """获取当前PDF文件的元数据"""
    metadata = {"pdf_files": []}
    try:
        for file_path in pdf_files:
            stat = os.stat(file_path)
            metadata["pdf_files"].append({
                "path": file_path,
                "mtime": stat.st_mtime,
                "size": stat.st_size
            })
        return metadata
    except Exception as e:
        logger.error(f"获取文件元数据失败: {str(e)}")
        return None

def load_cache(cache_path: str, current_metadata: dict) -> list:
    """尝试加载缓存文档"""
    if not os.path.exists(cache_path):
        return None
    
    try:
        with open(cache_path, "rb") as f:
            cached_data = pickle.load(f)
    except Exception as e:
        logger.warning(f"缓存加载失败: {str(e)}")
        return None

    # 元数据对比校验
    cached_metadata = cached_data.get("metadata", {})
    if cached_metadata.get("pdf_files", []) != current_metadata.get("pdf_files", []):
        return None
    
    logger.info(f"✅ 从缓存加载 {len(cached_data['docs'])} 个文档")
    return cached_data["docs"]

def save_cache(cache_path: str, docs: list, metadata: dict):
    """保存处理结果到缓存"""
    try:
        with open(cache_path, "wb") as f:
            pickle.dump({
                "metadata": metadata,
                "docs": docs
            }, f)
        logger.info(f"缓存已保存至 {cache_path}")
    except Exception as e:
        logger.error(f"缓存保存失败: {str(e)}")

def monitor_load_pdfs(folder_path: str) -> list:
    """带缓存机制的PDF加载函数"""
    # 获取所有PDF文件
    pdf_files = [
        str(p) for p in Path(folder_path).glob("**/*") 
        if p.suffix.lower() == ".pdf"
    ]
    
    if not pdf_files:
        logger.warning("目录中没有找到PDF文件")
        return []

    # 缓存机制
    cache_path = get_cache_path(folder_path)
    current_metadata = get_current_metadata(pdf_files)
    
    # 尝试加载缓存
    if current_metadata:
        cached_docs = load_cache(cache_path, current_metadata)
        if cached_docs:
            return cached_docs

    # 无有效缓存时重新处理
    total_files = len(pdf_files)
    progress_bar = tqdm(
        total=total_files,
        desc="📂 加载PDF文件",
        unit="file",
        bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]"
    )

    all_docs = []
    for file_path in pdf_files:
        try:
            logger.info(f"▶ 开始处理: {os.path.basename(file_path)}")
            start_time = time.time()

            loader = PyPDFLoader(file_path)
            docs = loader.load()

            duration = time.time() - start_time
            logger.debug(f"✓ 完成加载: {os.path.basename(file_path)} "
                        f"({len(docs)}页, 耗时{duration:.2f}s)")
            
            all_docs.extend(docs)
            progress_bar.update(1)
            progress_bar.set_postfix_str(f"当前: {os.path.basename(file_path)}")

        except Exception as e:
            logger.error(f"✕ 加载失败: {os.path.basename(file_path)} - {str(e)}", 
                       exc_info=True)
            progress_bar.set_postfix_str(f"失败: {os.path.basename(file_path)}", 
                                       refresh=True)
        finally:
            progress_bar.refresh()

    progress_bar.close()
    logger.info(f"✅ 完成所有加载 - 成功 {len(all_docs)/len(pdf_files):.0%} "
               f"({len(all_docs)}文档)")

    # 保存新缓存
    if current_metadata:
        save_cache(cache_path, all_docs, current_metadata)
    else:
        logger.warning("未能生成有效元数据，跳过缓存保存")

    return all_docs

# 使用示例
pdf_folder = r'C:\Users\lenovo\Documents\industry report'
docs = monitor_load_pdfs(pdf_folder)
print(docs[:10])

2025-03-21 23:39:18,358 [INFO] ✅ 从缓存加载 43791 个文档
[Document(metadata={'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'creator': '', 'creationdate': '', 'moddate': '2025-02-16T21:12:44+08:00', 'title': 'AIoT星图研究院-仓储行业：中国智能仓储市场调研报告(2025)-250214.pdf', 'author': '', 'subject': '', 'keywords': '', 'source': 'C:\\Users\\lenovo\\Documents\\industry report\\H3_AP202502161643146491_1.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}, page_content=''), Document(metadata={'producer': 'Aspose.PDF for .NET 19.1; modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'creator': '', 'creationdate': '', 'moddate': '2025-02-16T21:12:44+08:00', 'title': 'AIoT星图研究院-仓储行业：中国智能仓储市场调研报告(2025)-250214.pdf', 'author': '', 'subject': '', 'keywords': '', 'source': 'C:\\Users\\lenovo\\Documents\\industry report\\H3_AP202502161643146491_1.pdf', 'total_pages': 38, 'page': 1, 'page_label': '2'}, page_content='\x13\n本报告是 AIoT 星图研究院和深圳市物联传媒有限公司的调研与研究成果。 本报告内所有数据、 观点、\n结论的版权均属 AIOT

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from tqdm import tqdm
import os
import re

def clean_text(text):
    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"[^\w\s\u4e00-\u9fff，。！？；：“”‘’（）《》【】]", "", text)
    return text.strip()

def get_vectorstore(docs, embeddings_path="C:\\Users\\lenovo\\bge-large-zh-v1.5", 
                   save_path="C:\\Users\\lenovo\\Documents\\industry report\\faiss_index", 
                   force_rebuild=False):
    # 初始化嵌入模型
    model_kwargs = {'trust_remote_code': True}
    encode_kwargs = {
        "normalize_embeddings": True,
        "batch_size": 32,
        "truncation": True,
        "padding": "max_length",
        "max_length": 512
    }
    embeddings = HuggingFaceEmbeddings(
        model_name=embeddings_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    # 尝试加载已有索引
    if not force_rebuild and os.path.exists(save_path):
        try:
            print(f"加载本地索引: {save_path}")
            return FAISS.load_local(save_path, embeddings, allow_dangerous_deserialization=True)
        except Exception as e:
            print(f"加载失败: {str(e)}，重建索引")

    # 清洗文档
    valid_docs = []
    for i, doc in enumerate(docs):
        if isinstance(doc, Document) and isinstance(doc.page_content, str):
            cleaned = clean_text(doc.page_content)
            if len(cleaned) > 10:  # 至少保留10个字符
                doc.page_content = cleaned
                valid_docs.append(doc)
            else:
                print(f"文档 {i} 内容过短")
        else:
            print(f"无效文档 {i}")
    print(f"有效文档: {len(valid_docs)}/{len(docs)}")

    # 分批次处理
    batch_size = 100
    batches = [valid_docs[i:i + batch_size] for i in range(0, len(valid_docs), batch_size)]
    
    vector_store = None
    for batch in tqdm(batches, desc="处理批次"):
        try:
            batch_store = FAISS.from_documents(batch, embeddings)
            if vector_store is None:
                vector_store = batch_store
            else:
                vector_store.merge_from(batch_store)
        except Exception as e:
            print(f"批次处理失败: {str(e)}")
            # 打印前3个文档内容辅助调试
            for idx, doc in enumerate(batch[:3]):
                print(f"问题文档 {idx}: {doc.page_content[:200]}...")
            continue

    # 保存索引
    if vector_store is not None:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        vector_store.save_local(save_path)
        print(f"索引保存至: {save_path}")
    else:
        raise ValueError("未成功构建向量存储，请检查输入数据")
    return vector_store

# 使用示例
vectorStoreDB = get_vectorstore(docs)

# 创建检索器
retriever2 = vectorStoreDB.as_retriever(
    search_type='mmr',
    search_kwargs={"k":60}
)

result = retriever2.get_relevant_documents("电力行业怎么样")

# 打印查询结果
print("\n查询结果:")
for doc in result:
    print(doc.page_content)

2025-03-21 23:39:40,293 [INFO] Use pytorch device_name: cpu
2025-03-21 23:39:40,294 [INFO] Load pretrained SentenceTransformer: C:\Users\lenovo\bge-large-zh-v1.5
文档 0 内容过短
文档 419 内容过短
文档 485 内容过短
文档 486 内容过短
文档 487 内容过短
文档 488 内容过短
文档 489 内容过短
文档 490 内容过短
文档 491 内容过短
文档 492 内容过短
文档 493 内容过短
文档 494 内容过短
文档 495 内容过短
文档 496 内容过短
文档 497 内容过短
文档 498 内容过短
文档 499 内容过短
文档 500 内容过短
文档 501 内容过短
文档 502 内容过短
文档 503 内容过短
文档 504 内容过短
文档 505 内容过短
文档 506 内容过短
文档 507 内容过短
文档 508 内容过短
文档 509 内容过短
文档 510 内容过短
文档 511 内容过短
文档 512 内容过短
文档 513 内容过短
文档 514 内容过短
文档 515 内容过短
文档 516 内容过短
文档 517 内容过短
文档 518 内容过短
文档 519 内容过短
文档 520 内容过短
文档 521 内容过短
文档 522 内容过短
文档 523 内容过短
文档 524 内容过短
文档 525 内容过短
文档 530 内容过短
文档 531 内容过短
文档 532 内容过短
文档 534 内容过短
文档 535 内容过短
文档 536 内容过短
文档 691 内容过短
文档 692 内容过短
文档 693 内容过短
文档 694 内容过短
文档 695 内容过短
文档 696 内容过短
文档 697 内容过短
文档 698 内容过短
文档 699 内容过短
文档 700 内容过短
文档 701 内容过短
文档 702 内容过短
文档 703 内容过短
文档 706 内容过短
文档 707 内容过短
文档 708 内容过短
文档 709 内容过短
文档 710 内容过短
文档 722 内容过短
文档 723 内容过短
文档 726 内容过短


处理批次:   0%|                                                                                | 0/408 [00:00<?, ?it/s]

2025-03-21 23:43:31,966 [INFO] Loading faiss with AVX2 support.
2025-03-21 23:43:32,091 [INFO] Successfully loaded faiss with AVX2 support.
2025-03-21 23:43:32,109 [INFO] Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes.


处理批次: 100%|██████████████████████████████████████████████████████████████████| 408/408 [34:45:22<00:00, 306.67s/it]


索引保存至: C:\Users\lenovo\Documents\industry report\faiss_index


C:\Users\lenovo\AppData\Local\Temp\ipykernel_2896\2787370436.py:90: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = retriever2.get_relevant_documents("电力行业怎么样")



查询结果:
英大证券研究所证券研究报告
行 业 研 究
请务必阅读最后一页的免责条款 1
电力能源行业
2025年3月3日
投资评级：强于大市
最近一年走势
数据来源：iFind英大证券研究所
分析师：游虎
执业证书编号：S0990523060001
电话：075583007019
邮箱：youhuydzqsgcccomcn
相关报告
电力能源行业周报（20252172025223）
电力能源行业周报（20252102025216）
电力能源行业周报（202523202529）
电力能源行业周报（20251202025127）
电力能源行业周报（20251132025119）
电力能源行业周报（2025162025112）
电力能源行业周报（20241230202415）
电力能源行业周报（2024122320241229）
电力能源行业周报（2024121620241222）
电力能源行业周报（202412920241215）
电力能源行业周报（20241222024128）
电力能源行业周报（202411252024121）
电力能源行业周报（2024111820241124）
电力能源行业周报（2025224202532）
行业周报
 指数表现：根据iFind数据，20252242025228期间，沪深300下跌222，
申万电力设备指数下跌 083，跑赢沪深 300指数 140pct。
 行业表现：根据 iFind 数据，20252242025228 期间，31 个申万一级行
业中，电力设备下跌 083，排第 15位。申万三级行业，电力能源相关子板
块中，蓄电池及其他电池光伏辅材光伏电池组件涨跌幅位列前三位，分
别上涨 832上涨 493上涨 283；电机Ⅲ电工仪器仪表锂电池涨
跌幅位列后三位，分别下跌 422下跌 395下跌 305。
 电力工业运行：根据能源局数据，2024 年 12 月全社会用电量为 8835亿千瓦
时，同比增长 327；2024年 112月全社会用电量累计为 98521亿千瓦时，
同比增长 681。2024年 112 月，新增发电装机容量 43323 万千瓦，同比增
长 1738；2024 年 112 月，发电设备平均利用小时 3442 小时，同比减少
150 小时；2024 年 112 月电网累计投资 6083 亿元，同比增长 1

In [9]:
import akshare as ak
import numpy as np
industry_code=[
    "801010","801030","801040",
"801050","801080","801110","801120","801130","801140","801150",
"801160","801170","801180",
"801200","801210","801230","801710",
"801720","801730","801740","801750",
"801760","801770","801780",
"801790","801880","801890","801950","801960"
,"801970","801980"
]
def macd(col,short=12,long=26,mid=9):
    DIF=col.ewm(span=short, adjust=False).mean()-col.ewm(span=long, adjust=False).mean()
    DEA=DIF.ewm(span=mid, adjust=False).mean()
    MACD=(DIF-DEA)*2
    return MACD

def rsi(col, window=14):
    delta = col.diff(1)  # 计算价格变动
    gain = delta.where(delta > 0, 0)  # 分离涨幅
    loss = -delta.where(delta < 0, 0)  # 分离跌幅（取绝对值）

    # 计算初始平均（前window天简单平均）
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    # 计算后续平滑平均（Wilder's Smoothing）
    for i in range(window, len(gain)):
        avg_gain[i] = (avg_gain[i-1] * (window-1) + gain[i]) / window
        avg_loss[i] = (avg_loss[i-1] * (window-1) + loss[i]) / window

    # 计算RS和RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def boll(col,n=20, k=2):
        # 计算移动平均线(中轨)
        middle = col.rolling(window=n).mean()
        
        # 计算标准差
        std = col.rolling(window=n).std()
        
        # 计算上轨和下轨
        upper = middle + k * std
        lower = middle - k * std
        
        # 计算因子值：价格在布林带中的相对位置
        boll= np.clip((col - lower) / (upper - lower),0,1)
        return boll

for code in industry_code: 
    index_hist_sw_df = ak.index_hist_sw(symbol=code, period="day")
    print(index_hist_sw_df)
    index_hist_sw_df["MA_5"] = index_hist_sw_df["收盘"].pct_change(periods=5) 
    index_hist_sw_df["MA_20"] = index_hist_sw_df["收盘"].pct_change(periods=20) 
    index_hist_sw_df["MA_60"] = index_hist_sw_df["收盘"].pct_change(periods=60) 
    index_hist_sw_df["MA_120"] = index_hist_sw_df["收盘"].pct_change(periods=120) 
    index_hist_sw_df["EMA_5"] = index_hist_sw_df["收盘"].ewm(span=5, adjust=False).mean()
    index_hist_sw_df["EMA_20"] = index_hist_sw_df["收盘"].ewm(span=20, adjust=False).mean()
    index_hist_sw_df["EMA_60"] = index_hist_sw_df["收盘"].ewm(span=60, adjust=False).mean()
    index_hist_sw_df["EMA_120"] = index_hist_sw_df["收盘"].ewm(span=120, adjust=False).mean()
    index_hist_sw_df["MACD"] = macd(index_hist_sw_df["收盘"])
    index_hist_sw_df["RSI"] = rsi(index_hist_sw_df["收盘"])
    index_hist_sw_df["BOLL"] = boll(index_hist_sw_df["收盘"])
    print(index_hist_sw_df)
    index_hist_sw_df.to_csv(rf"C:\Users\lenovo\Documents\industry price\{code}.csv", index=False, encoding="utf-8-sig") 

          代码          日期       收盘       开盘       最高       最低        成交量  \
0     801010  1999-12-30  1000.00  1000.00  1000.00  1000.00   0.098647   
1     801010  2000-01-04  1027.66  1001.98  1035.36   985.47   0.140116   
2     801010  2000-01-05  1028.87  1035.47  1057.85  1013.07   0.253309   
3     801010  2000-01-06  1065.67  1025.38  1070.79  1011.00   0.302028   
4     801010  2000-01-07  1106.19  1076.06  1117.71  1058.83   0.648305   
...      ...         ...      ...      ...      ...      ...        ...   
6088  801010  2025-03-17  2579.56  2571.36  2596.01  2571.36  26.766366   
6089  801010  2025-03-18  2562.48  2578.50  2578.50  2555.04  20.833735   
6090  801010  2025-03-19  2556.20  2560.55  2560.79  2548.67  17.179189   
6091  801010  2025-03-20  2547.26  2560.55  2562.66  2544.84  17.526624   
6092  801010  2025-03-21  2532.67  2540.72  2564.77  2516.40  23.670651   

             成交额  
0       1.239666  
1       1.831998  
2       2.904446  
3       4.214445  
4   

In [25]:
from langchain_community.document_loaders import CSVLoader
import pickle
import os
from tqdm import tqdm
import glob

# 配置参数
cache_path = 'C:/Users/lenovo/Documents/industry price/industry_price_docs.pkl'
data_path = 'C:/Users/lenovo/Documents/industry price/**/*.csv'

def load_with_progress():
    """带详细错误处理的加载函数"""
    print("🕵️ 扫描CSV文件中...")
    file_paths = glob.glob(data_path, recursive=True)
    
    # 过滤无效路径
    valid_files = [fp for fp in file_paths if os.path.exists(fp)]
    if not valid_files:
        raise FileNotFoundError("未找到任何有效CSV文件")
    print(f"📂 发现 {len(valid_files)} 个有效文件")

    all_docs = []
    failed_files = []
    
    for file_path in tqdm(valid_files, desc="加载进度"):
        try:
            # 关键修改点：指定编码和解析参数
            loader = CSVLoader(
                file_path=file_path,
                encoding="utf-8-sig",  # 优先尝试中文编码
                csv_args={
                    "delimiter": ",",
                    "quotechar": '"'
                }
            )
            docs = loader.load()
            all_docs.extend(docs)
        except Exception as e:
            failed_files.append(file_path)
            error_msg = f"\n⚠️ 加载失败 [{type(e).__name__}]: {file_path}\n    错误详情: {str(e)}"
            print(error_msg)

    # 输出统计信息
    print("\n" + "="*50)
    print(f"✅ 成功加载: {len(all_docs)} 个文档")
    print(f"❌ 失败文件: {len(failed_files)} 个")
    if failed_files:
        print("失败列表:\n" + "\n".join(failed_files))
    return all_docs

# 其余缓存逻辑保持不变...

# 主程序
if os.path.exists(cache_path):
    print("🔍 检测到缓存文件")
    try:
        print("📥 正在加载缓存...")
        with open(cache_path, 'rb') as f:
            docs3 = pickle.load(f)
        print(f"♻️ 从缓存成功加载 {len(docs3)} 个文档")
    except Exception as e:
        print(f"❌ 缓存加载失败: {str(e)}，尝试重新生成...")
        docs3 = load_with_progress()
else:
    print("⏳ 未找到缓存文件，开始初始化加载")
    docs3 = load_with_progress()
    
    print("💾 正在保存缓存...")
    with open(cache_path, 'wb') as f:
        pickle.dump(docs3, f)
    print(f"📁 缓存已保存到 {cache_path}")

# 最终结果输出
print("="*50)
print(f"当前文档总数: {len(docs3)}")
print(f"前十条文档: {docs3[:10]}")
print("="*50)

🔍 检测到缓存文件
📥 正在加载缓存...
♻️ 从缓存成功加载 130807 个文档
当前文档总数: 130807
前十条文档: [Document(metadata={'source': 'C:/Users/lenovo/Documents/industry price\\801010.csv', 'row': 0}, page_content='代码: 801010\n日期: 1999-12-30\n收盘: 1000.0\n开盘: 1000.0\n最高: 1000.0\n最低: 1000.0\n成交量: 0.09864744\n成交额: 1.2396662266\nMA_5: \nMA_20: \nMA_60: \nMA_120: \nEMA_5: 1000.0\nEMA_20: 1000.0\nEMA_60: 1000.0\nEMA_120: 1000.0\nMACD: 0.0\nRSI: \nBOLL: '), Document(metadata={'source': 'C:/Users/lenovo/Documents/industry price\\801010.csv', 'row': 1}, page_content='代码: 801010\n日期: 2000-01-04\n收盘: 1027.66\n开盘: 1001.98\n最高: 1035.36\n最低: 985.47\n成交量: 0.14011599\n成交额: 1.831998425\nMA_5: \nMA_20: \nMA_60: \nMA_120: \nEMA_5: 1009.22\nEMA_20: 1002.6342857142857\nEMA_60: 1000.9068852459017\nEMA_120: 1000.4571900826446\nMACD: 3.530393162393011\nRSI: \nBOLL: '), Document(metadata={'source': 'C:/Users/lenovo/Documents/industry price\\801010.csv', 'row': 2}, page_content='代码: 801010\n日期: 2000-01-05\n收盘: 1028.87\n开盘: 1035.47\n最高: 1057.85\n最低:

In [37]:
import os
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_path="C:\\Users\\lenovo\\bge-large-zh-v1.5"
model_kwargs = {'trust_remote_code': True}
embeddings=HuggingFaceEmbeddings(model_name=embeddings_path,model_kwargs=model_kwargs)

# 配置参数
faiss_index_path = "C:/Users/lenovo/Documents/industry price/faiss_index"  # 索引保存路径
chunk_size = 100  # 处理批次大小

def build_faiss_with_progress(documents, embeddings):
    """带进度条的FAISS索引构建函数"""
    if not documents:
        raise ValueError("文档列表不能为空")
    
    print("🛠️ 开始构建FAISS索引...")
    
    # 初始化进度条
    pbar = tqdm(total=len(documents), desc="处理文档", unit="doc")
    
    try:
        # 分批次构建索引
        vector_store = FAISS.from_documents(documents[:chunk_size], embeddings)
        pbar.update(chunk_size)
        
        # 剩余文档分批次添加
        for i in range(chunk_size, len(documents), chunk_size):
            batch = documents[i:i+chunk_size]
            vector_store.add_documents(batch)
            pbar.update(len(batch))
            
        return vector_store
    finally:
        pbar.close()

# 主程序
if os.path.exists(os.path.join(faiss_index_path, "index.faiss")):
    print("🔍 检测到已有FAISS索引")
    try:
        # 加载现有索引
        print("⏳ 正在加载索引...")
        vectorStoreDB3 = FAISS.load_local(
            faiss_index_path,
            embeddings,
            allow_dangerous_deserialization=True  # 需要确认文件安全性
        )
        print(f"✅ 成功加载已有索引（包含 {len(vectorStoreDB3.docstore._dict)} 个文档）")
    except Exception as e:
        print(f"❌ 索引加载失败: {str(e)}")
        print("🔄 正在重新构建索引...")
        vectorStoreDB3 = build_faiss_with_progress(docs3, embeddings)
        vectorStoreDB3.save_local(faiss_index_path)
else:
    print("⏳ 未找到现有索引，开始构建...")
    os.makedirs(faiss_index_path, exist_ok=True)
    vectorStoreDB3 = build_faiss_with_progress(docs3, embeddings)
    vectorStoreDB3.save_local(faiss_index_path)
    print(f"💾 索引已保存到 {faiss_index_path}")

# 创建检索器
retriever3 = vectorStoreDB3.as_retriever(
    search_type='mmr',
    search_kwargs={"k": 60}
)

print("\n" + "="*50)
print(f"当前索引文档数: {len(vectorStoreDB3.docstore._dict)}")
print("="*50)

result = retriever3.get_relevant_documents("801890怎么样")

# 打印查询结果
print("\n查询结果:")
for doc in result:
    print(doc.page_content)

2025-03-25 16:08:23,704 [INFO] Use pytorch device_name: cpu
2025-03-25 16:08:23,705 [INFO] Load pretrained SentenceTransformer: C:\Users\lenovo\bge-large-zh-v1.5
🔍 检测到已有FAISS索引
⏳ 正在加载索引...
✅ 成功加载已有索引（包含 130807 个文档）

当前索引文档数: 130807

查询结果:
代码: 801890
日期: 2015-08-19
收盘: 1989.33
开盘: 1886.79
最高: 2002.27
最低: 1830.34
成交量: 46.42663506
成交额: 717.6003167133
MA_5: -0.027298597664730417
MA_20: -0.03815786292759582
MA_60: -0.2695766156421997
MA_120: 0.3490367075130709
EMA_5: 2018.7736259103704
EMA_20: 2010.9742511543789
EMA_60: 2111.2773314824485
EMA_120: 2047.9444377300954
MACD: 39.61145815901189
RSI: 47.39981371506783
BOLL: 0.5170020884662994
代码: 801890
日期: 2018-09-19
收盘: 1035.89
开盘: 1022.74
最高: 1041.41
最低: 1021.58
成交量: 20.76775121
成交额: 160.9307987418
MA_5: 0.019847794197277002
MA_20: -0.004870504150015331
MA_60: -0.0609878803810836
MA_120: -0.216593939302271
EMA_5: 1023.4078800723446
EMA_20: 1032.744723233218
EMA_60: 1081.2117246609894
EMA_120: 1156.6859640397727
MACD: 4.463085847310367
RSI: 48.

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
#from langchain.out_parser import XMLOutputParser
from langchain_openai import ChatOpenAI
import os
from operator import itemgetter
deepseek_key=os.getenv("DEEPSEEK_API_KEY")
llm=ChatOpenAI(temperature=0.95,model="deepseek-chat",api_key=deepseek_key,base_url="https://api.deepseek.com")

industry=[
    "农林牧渔","基础化工","钢铁","有色金属",
"电子","家用电器","食品饮料","纺织服饰",
"轻工制造","医药生物","公用事业","交通运输",
"房地产","商贸零售","社会服务","综合",
"建筑材料","建筑装饰","电力设备","国防军工",
"计算机","传媒","通信","银行","非银金融",
"汽车","机械设备","煤炭","石油石化","环保","美容护理"
]

industry_map={
    "801010":"农林牧渔",
    "801030":"基础化工",
    "801040":"钢铁",
    "801050":"有色金属",
    "801080":"电子",
    "801110":"家用电器",
    "801120":"食品饮料",
    "801130":"纺织服饰",
    "801140":"轻工制造",
    "801150":"医药生物",
    "801160":"公用事业",
    "801170":"交通运输",
    "801180":"房地产",
    "801200":"商贸零售",
    "801210":"社会服务",
    "801230":"综合",
    "801710":"建筑材料",
    "801720":"建筑装饰",
    "801730":"电力设备",
    "801740":"国防军工",
    "801750":"计算机",
    "801760":"传媒",
    "801770":"通信",
    "801780":"银行",
    "801790":"非银金融",
    "801880":"汽车",
    "801890":"机械设备",
    "801950":"煤炭",
    "801960":"石油石化",
    "801970":"环保",
    "801980":"美容护理"
    }

template="""
"请根据以下新闻信息{context}和以下研报信息{report}和行情数据{price}中的开盘、最高、最低、成交量、成交额、MA_5、MA_20、MA_60、MA_120、EMA_5、EMA_20、EMA_60、EMA_120、MACD、RSI、BOLL回答问题：
行业：{industry}
行情数据{price}中代码和行业的对应关系为:{industry_map}
问题：{question}
"""

prompt=ChatPromptTemplate.from_template(template)

outputParser=StrOutputParser()
#parser=XMLOutputParser()

chain = (
    {
        "context": itemgetter("question") | retriever,
        "report": itemgetter("question") | retriever2,
        "price": itemgetter("question") | retriever3,
        "question": itemgetter("question"),
        "industry": itemgetter("industry"),
        "industry_map":itemgetter("industry_map")
    }
    | prompt
    | llm
    | outputParser
)


result=chain.invoke({"question":"接下来一个月要投资什么行业并说明理由","industry":industry,"industry_map":industry_map})
print("结果：",result)

2025-03-25 16:20:59,704 [INFO] HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
结果： 根据提供的新闻信息、研报内容和行情数据，结合当前政策导向和行业趋势，以下是对接下来一个月（2025年3月）的行业投资建议及分析：

### **推荐投资行业及理由**
#### 1. **农林牧渔（801010）**
- **政策支持**：2025年中央一号文件强调粮食安全和农业新质生产力，政策红利持续释放（东兴证券研报）。国家发改委提及创业投资引导基金将带动社会资本，农业现代化领域可能受益。
- **行情指标**：  
  - **MA_5（0.018）和MA_20（0.090）**显示短期和中期均线呈上升趋势，技术面偏强。  
  - **RSI（61.29）**处于健康区间，无超买压力。  
  - **MACD（11.24）**金叉向上，动能增强。  
- **催化剂**：春耕备耕季节临近，农业需求季节性回升，叠加政策落地预期。

#### 2. **房地产（801180）**
- **政策催化**：  
  - 政府工作报告首次明确“稳楼市”，多地专项债收储土地（国金证券研报）。  
  - 育儿补贴等政策刺激改善型需求，3月“小阳春”新房供应倍增（国金证券）。  
- **行情指标**：  
  - **MA_60（0.017）**转正，显示长期趋势改善。  
  - **成交量与成交额**近期放大（如2月数据），市场活跃度提升。  
- **风险提示**：需关注居民中长期贷款数据是否持续回升。

#### 3. **国防军工（801740）**
- **政策与事件驱动**：  
  - 国防预算7.2%稳健增长，确保“十四五”规划收官（国金证券）。  
  - 军工AI加速发展（如DeepSeek技术突破），军事智能化主题升温。  
- **技术面**：  
  - **BOLL（0.637）**接近中轨，存在上行空间。  
  - **RSI（59.24）**中性偏强，未达超买。  

#### 4. **机械设备（801890）**
- **新质生产力主题**：  
  - 国家创业投资引导基金带动万亿资金，工程机械、人形机